# DeepLIFT on Tox21 dataset for GCN

In [1]:
from dig.xgraph.dataset import MoleculeDataset
from dig.xgraph.models import GCN_3l
import torch
from torch.utils.data import random_split
from torch_geometric.data import DataLoader
from torch_geometric.data import Data, InMemoryDataset, download_url, extract_zip
from torch_geometric.data.dataset import files_exist
import os.path as osp
import os

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Load dataset

In [2]:
def split_dataset(dataset, dataset_split=[0.8, 0.1, 0.1]):
    dataset_len = len(dataset)
    dataset_split = [int(dataset_len * dataset_split[0]),
                     int(dataset_len * dataset_split[1]),
                     0]
    dataset_split[2] = dataset_len - dataset_split[0] - dataset_split[1]
    train_set, val_set, test_set = \
        random_split(dataset, dataset_split)

    return {'train': train_set, 'val': val_set, 'test': test_set}

dataset = MoleculeDataset('datasets', 'Tox21')
dataset.data.x = dataset.data.x.to(torch.float32)
dataset.data.y = dataset.data.y[:, 2] # the target 2 task.
dim_node = dataset.num_node_features
dim_edge = dataset.num_edge_features
num_targets = dataset.num_classes
num_classes = 2

splitted_dataset = split_dataset(dataset)
dataloader = DataLoader(splitted_dataset['test'], batch_size=1, shuffle=False)

### Load model and checkpoints

In [3]:
def check_checkpoints(root='./'):
    if osp.exists(osp.join(root, 'checkpoints')):
        return
    url = ('https://github.com/divelab/DIG_storage/raw/main/xgraph/checkpoints.zip')
    path = download_url(url, root)
    extract_zip(path, root)
    os.unlink(path)

model = GCN_3l(model_level='graph', dim_node=dim_node, dim_hidden=300, num_classes=num_classes)
model.to(device)
check_checkpoints()
ckpt_path = osp.join('checkpoints', 'tox21', 'GCN_3l', '2', 'GCN_3l_best.ckpt')
model.load_state_dict(torch.load(ckpt_path)['state_dict'])

<All keys matched successfully>

### Display example output

In [4]:
data = list(dataloader)[0].to(device)
out = model(data.x, data.edge_index)
print(out)

tensor([[ 1.0259, -1.0004]], device='cuda:0', grad_fn=<AddmmBackward>)


### Load the explainer

In [5]:
from dig.xgraph.method import DeepLIFT
explainer = DeepLIFT(model, explain_graph=True)

### Setup for evaluation

In [6]:
# --- Set the Sparsity to 0.5 ---
sparsity = 0.5

# --- Create data collector and explanation processor ---
from dig.xgraph.evaluation import XCollector, ExplanationProcessor
x_collector = XCollector(sparsity)
# x_processor = ExplanationProcessor(model=model, device=device)

### Run explainer on the given model and dataset

In [7]:
for index, data in enumerate(dataloader):
    print(f'explain graph line {dataloader.dataset.indices[index] + 2}')
    data.to(device)

    if torch.isnan(data.y[0].squeeze()):
        continue

    walks, masks, related_preds = \
        explainer(data.x, data.edge_index, sparsity=sparsity, num_classes=num_classes)

    x_collector.collect_data(masks, related_preds, data.y[0].squeeze().long().item())

    # if you only have the edge masks without related_pred, please feed sparsity controlled mask to
    # obtain the result: x_processor(data, masks, x_collector)

    if index >= 99:
        break

explain graph line 7545
explain graph line 7711
explain graph line 4137
explain graph line 1463
explain graph line 4455
explain graph line 4251
explain graph line 6633
explain graph line 3725
explain graph line 282
explain graph line 686
explain graph line 209
explain graph line 7748
explain graph line 5089
explain graph line 5055
explain graph line 424
explain graph line 1651
explain graph line 1513
explain graph line 7199
explain graph line 2637
explain graph line 3348
explain graph line 4963
explain graph line 5747
explain graph line 5226
explain graph line 5231
explain graph line 506
explain graph line 2950
explain graph line 5466
explain graph line 4824
explain graph line 2036
explain graph line 451
explain graph line 524
explain graph line 4899
explain graph line 556
explain graph line 3015
explain graph line 2414
explain graph line 3659
explain graph line 5259
explain graph line 6400
explain graph line 1288
explain graph line 5436
explain graph line 3796
explain graph line 1621


### Output metrics evaluation results

In [8]:
print(f'Fidelity: {x_collector.fidelity:.4f}\n'
      f'Fidelity_inv: {x_collector.fidelity_inv:.4f}\n'
      f'Sparsity: {x_collector.sparsity:.4f}')

Fidelity: 0.1843
Fidelity_inv: -0.0963
Sparsity: 0.5000
